# Telescope AWS Local Development

This notebook provides a step-by-step guide for manually running Terraform for AWS telescope testing.

## Prerequisites

Before running this notebook, ensure you have:
* [Terraform 1.7.3](https://developer.hashicorp.com/terraform/tutorials/aws-get-started/install-cli)
* [AWS CLI v2](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html)
* [jq 1.6-2.1ubuntu3](https://stedolan.github.io/jq/download/)

**Note**: All commands should be run from the root of the repository in a bash shell (Linux or WSL).

## 1. Prerequisites Check

First, let's verify that all required tools are installed and accessible.

In [1]:
# Check prerequisites
echo "🔍 Checking prerequisites..."
echo "==========================================="

# Check Terraform
if command -v terraform &> /dev/null; then
    TERRAFORM_VERSION=$(terraform version -json | jq -r '.terraform_version')
    echo "✅ Terraform: $TERRAFORM_VERSION"
else
    echo "❌ Terraform not found"
fi

# Check AWS CLI
if command -v aws &> /dev/null; then
    AWS_VERSION=$(aws --version 2>&1)
    echo "✅ AWS CLI: $AWS_VERSION"
else
    echo "❌ AWS CLI not found"
fi

# Check jq
if command -v jq &> /dev/null; then
    JQ_VERSION=$(jq --version)
    echo "✅ jq: $JQ_VERSION"
else
    echo "❌ jq not found"
fi

echo ""
echo "📍 Current directory: $(pwd)"
echo "💡 Make sure you're running from the telescope repository root"

# Navigate to repository root if not already there
if [ ! -f "scenarios" ] && [ ! -d "scenarios" ]; then
    echo "🔄 Navigating to repository root..."
    cd ../../../
    echo "📍 New directory: $(pwd)"
    export ROOT_DIR=$(pwd)
fi

🔍 Checking prerequisites...
✅ Terraform: 1.13.1
✅ AWS CLI: aws-cli/2.28.19 Python/3.13.7 Linux/6.11.0-1018-azure exe/x86_64.ubuntu.24
✅ jq: jq-1.7

📍 Current directory: /home/schinnapulla/aks/telescope/modules/terraform/aws
💡 Make sure you're running from the telescope repository root
🔄 Navigating to repository root...
📍 New directory: /home/schinnapulla/aks/telescope


## 2. Define Variables

Set environment variables for your specific test scenario. This example uses `perf-eval/nap` scenario.

In [ ]:

# Define test scenario variables
export SCENARIO_TYPE=perf-eval
export SCENARIO_NAME=nap
export OWNER=$(whoami)
export RUN_ID=$(date +%s)
export CLOUD=aws
export REGION=us-east-2
export K8S_MACHINE_TYPE=""
export K8S_OS_DISK_TYPE=""
export CREATION_TIME=$(date -uIseconds | sed 's/+00:00/Z/')
export TERRAFORM_MODULES_DIR=modules/terraform/$CLOUD
export TERRAFORM_INPUT_FILE=$ROOT_DIR/scenarios/$SCENARIO_TYPE/$SCENARIO_NAME/terraform-inputs/${CLOUD}.tfvars
export TERRAFORM_USER_DATA_PATH=$ROOT_DIR/scenarios/$SCENARIO_TYPE/$SCENARIO_NAME/scripts/user_data

echo "📋 Configuration Summary:"
echo "========================"
echo "Scenario: $SCENARIO_TYPE/$SCENARIO_NAME"
echo "Owner: $OWNER"
echo "Run ID: $RUN_ID"
echo "Cloud: $CLOUD"
echo "Region: $REGION"
echo "K8S_MACHINE_TYPE: $K8S_MACHINE_TYPE"
echo "K8S_OS_DISK_TYPE: $K8S_OS_DISK_TYPE"
echo "Creation Time: $CREATION_TIME"
echo "Terraform Input File: $TERRAFORM_INPUT_FILE"
echo "User Data Path: $TERRAFORM_USER_DATA_PATH"
echo ""
echo "⚠️  Note: RUN_ID should be unique as it's used to name AWS resources"

📋 Configuration Summary:
Scenario: perf-eval/nap
Owner: schinnapulla
Run ID: 1757022761
Cloud: aws
Region: us-east-2
K8S_MACHINE_TYPE: 
K8S_OS_DISK_TYPE: 
Creation Time: 2025-09-04T21:52:41Z
Terraform Input File: /home/schinnapulla/aks/telescope/scenarios/perf-eval/nap/terraform-inputs/aws.tfvars
User Data Path: /home/schinnapulla/aks/telescope/scenarios/perf-eval/nap/scripts/user_data

⚠️  Note: RUN_ID should be unique as it's used to name AWS resources


## 3. AWS Authentication

Verify AWS credentials and configuration.

In [3]:
# AWS authentication check
echo "🔐 AWS Authentication"
echo "====================="

# Check if AWS credentials are configured
if aws sts get-caller-identity &> /dev/null; then
    echo "✅ AWS credentials are configured"
    
    # Display current identity
    echo "📋 Current AWS Identity:"
    aws sts get-caller-identity --output table    
else
    echo "❌ AWS credentials not configured or invalid"
    echo "💡 Please run 'aws configure' to set up your credentials"
    echo "   export AWS_ACCESS_KEY_ID=your-access-key"
    echo "   export AWS_SECRET_ACCESS_KEY=your-secret-key"
    echo "   export AWS_DEFAULT_REGION=$REGION"
fi

# Set the region for this session
export AWS_DEFAULT_REGION=$REGION
echo ""
echo "🔧 Session region set to: $AWS_DEFAULT_REGION"

🔐 AWS Authentication
✅ AWS credentials are configured
📋 Current AWS Identity:
------------------------------------------------------------
|                     GetCallerIdentity                    |
+---------+------------------------------------------------+
|  Account|  891516228446                                  |
|  Arn    |  arn:aws:iam::891516228446:user/schinnapulla   |
|  UserId |  AIDA47EUCCNPBVKRQOMLW                         |
+---------+------------------------------------------------+


🔧 Session region set to: us-east-2


## 4. Prepare Terraform Input JSON

Create the JSON input for Terraform with all the configuration parameters.

In [4]:
# Create INPUT_JSON variable for AWS
echo "📝 Preparing Terraform Input JSON"
echo "================================="

export INPUT_JSON=$(jq -n \
  --arg run_id $RUN_ID \
  --arg region $REGION \
  --arg creation_time "$CREATION_TIME" \
  --arg eks_kubernetes_version "$KUBERNETES_VERSION" \
  --arg k8s_machine_type "$K8S_MACHINE_TYPE" \
  --arg k8s_os_disk_type "$K8S_OS_DISK_TYPE" \
  --arg terraform_user_data_path "$TERRAFORM_USER_DATA_PATH" \
  '{
    run_id: $run_id,
    region: $region,
    creation_time: $creation_time,
    eks_kubernetes_version: $eks_kubernetes_version,
    k8s_machine_type: $k8s_machine_type,
    k8s_os_disk_type: $k8s_os_disk_type,
    terraform_user_data_path: $terraform_user_data_path
  }' | jq 'with_entries(select(.value != null and .value != ""))')

echo "📋 Terraform Input JSON:"
echo "$INPUT_JSON" | jq .

echo ""
echo "📂 Terraform Input File: $TERRAFORM_INPUT_FILE"
if [ -f "$TERRAFORM_INPUT_FILE" ]; then
    echo "✅ Terraform input file exists"
else
    echo "⚠️  Terraform input file not found"
fi


📝 Preparing Terraform Input JSON
📋 Terraform Input JSON:
{
  "run_id": "1757022761",
  "region": "us-east-2",
  "creation_time": "2025-09-04T21:52:41Z",
  "terraform_user_data_path": "/home/schinnapulla/aks/telescope/scenarios/perf-eval/nap/scripts/user_data"
}

📂 Terraform Input File: /home/schinnapulla/aks/telescope/scenarios/perf-eval/nap/terraform-inputs/aws.tfvars
✅ Terraform input file exists


## 5. Terraform Initialization & Plan

Initialize Terraform in the AWS modules directory.

In [5]:
# Terraform initialization
echo "🚀 Terraform Initialization"
echo "==========================="

# Change to terraform directory
pushd $TERRAFORM_MODULES_DIR

echo "📂 Current directory: $(pwd)"
echo "🔧 Initializing Terraform..."
terraform init
echo "🚀 Terraform Plan"
echo "==========================="
terraform plan \
  -var json_input="$(echo $INPUT_JSON | jq -c .)" \
  -var-file $TERRAFORM_INPUT_FILE

echo ""
popd

🚀 Terraform Initialization
~/aks/telescope/modules/terraform/aws ~/aks/telescope
📂 Current directory: /home/schinnapulla/aks/telescope/modules/terraform/aws
🔧 Initializing Terraform...
Initializing the backend...
Initializing modules...
Initializing provider plugins...
- terraform.io/builtin/terraform is built in to Terraform
- Reusing previous version of hashicorp/tls from the dependency lock file
- Reusing previous version of hashicorp/aws from the dependency lock file
- Using previously-installed hashicorp/tls v4.1.0
- Using previously-installed hashicorp/aws v6.2.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so i

## 7. Terraform Apply

Apply the Terraform configuration to provision AWS resources.

**⚠️ WARNING**: This will create actual AWS resources and may incur costs.

In [ ]:
# Terraform apply
echo "🚀 Terraform Apply"
echo "=================="

echo "⚠️  WARNING: This will provision actual AWS resources!"
echo "💰 This may incur costs in your AWS account"
echo "⏰ Resources will be tagged with creation time: $CREATION_TIME"
echo ""

pushd $TERRAFORM_MODULES_DIR
terraform apply -var json_input="$(echo $INPUT_JSON | jq -c .)" -var-file $TERRAFORM_INPUT_FILE --auto-approve
popd

🚀 Terraform Apply
⚠️  WARNING: This will provision actual AWS resources!
💰 This may incur costs in your AWS account
⏰ Resources will be tagged with creation time: 2025-09-04T21:52:41Z

~/aks/telescope/modules/terraform/aws ~/aks/telescope

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create
 <= read (data resources)

Terraform will perform the following actions:

  # module.eks["nap"].data.aws_ec2_instance_type.this["nap-c2n200p200-ng"] will be read during apply
  # (depends on a resource or a module with changes pending)
 <= data "aws_ec2_instance_type" "this" {
      + auto_recovery_supported              = (known after apply)
      + bandwidth_weightings                 = (known after apply)
      + bare_metal                           = (known after apply)
      + boot_modes                           = (known after apply)
      + burstable_performance_supported      = (known after appl

## 8. Cleanup Resources

**IMPORTANT**: Always clean up resources after testing to avoid unnecessary costs.

In [8]:
# Cleanup with Terraform
echo "🧹 Terraform Cleanup"
echo "====================="

echo "⚠️  WARNING: This will destroy all provisioned resources!"
echo "💾 Make sure to save any important data before proceeding"
echo ""

pushd $TERRAFORM_MODULES_DIR
terraform destroy -var json_input="$(echo $INPUT_JSON | jq -c .)" -var-file $TERRAFORM_INPUT_FILE --auto-approve
popd

🧹 Terraform Cleanup
⚠️  WARNING: This will destroy all provisioned resources!
💾 Make sure to save any important data before proceeding

bash: pushd: modules/terraform/aws: No such file or directory
module.virtual_network["nap"].aws_vpc.vpc: Refreshing state... [id=vpc-02b3cd12b1be2ec6d]
module.virtual_network["nap"].aws_internet_gateway.internet_gateway: Refreshing state... [id=igw-05ef225d5b74e1400]
module.virtual_network["nap"].aws_vpc_ipv4_cidr_block_association.secondary_ipv4_cidr_block["10.1.0.0/16"]: Refreshing state... [id=vpc-cidr-assoc-04813ca87b3796de2]
module.virtual_network["nap"].aws_security_group.security_group: Refreshing state... [id=sg-075092c4dca27baf0]
module.virtual_network["nap"].aws_subnet.subnets["nap-subnet-2"]: Refreshing state... [id=subnet-0212f08433d3778c4]
module.virtual_network["nap"].aws_subnet.subnets["nap-subnet-1"]: Refreshing state... [id=subnet-021ba52371aa75813]
module.virtual_network["nap"].aws_subnet.subnets["nap-subnet-3"]: Refreshing state... [